In [10]:
import xgboost as xgb
import pickle
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import pandas as pd
import dask.dataframe as dd

In [26]:
directory = '/mnt/home/jbielecki1/NEMA/10000000/'
max_depth = 0
n_estimators = 2

In [3]:
def loadData():
    global X_train, X_test, y_train, y_test, class_test, class_train
    X_train = dd.from_pandas(pickle.load(open(directory + 'xTrain', 'rb')), npartitions = 10)
    X_test = dd.from_pandas(pickle.load(open(directory + 'xTest', 'rb')), npartitions = 10)
    y_train = dd.from_pandas(pickle.load(open(directory + 'yTrain', 'rb')), npartitions = 10)
    y_test = dd.from_pandas(pickle.load(open(directory + 'yTest', 'rb')), npartitions = 10)
    class_test = y_test[["class"]].to_dask_array()
    class_train = y_train[["class"]].to_dask_array()
    y_train = y_train[['newClass']].to_dask_array()
    y_test = y_test[['newClass']].to_dask_array()

In [4]:
loadData()

In [27]:
model = xgb.XGBClassifier(
    objective = 'binary:logistic', # Logistic regression for binary classification, output probability
    booster = 'gbtree', # Set estimator as gradient boosting tree
    subsample = 1, # Percentage of the training samples used to train (consider this)
    n_estimators = n_estimators, # Number of trees in each classifier
    learning_rate = 0.2, # Contribution of each estimator
    max_depth = max_depth, # Maximum depth of a tree
    colsample_bytree = 0.6, # The fraction of columns to be subsampled
)

In [28]:
results = {}
eval_set = [( X_train, y_train), ( X_test, y_test)]
model.fit(
    X_train, y_train, 
    early_stopping_rounds = 20, 
    eval_set = eval_set,
    eval_metric = ["error"],
    callbacks = [xgb.callback.record_evaluation(results)]
)

/mnt/home/jbielecki1/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/mnt/home/jbielecki1/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/mnt/home/jbielecki1/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[23:59:34] Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	validation_0-error:0.292008	validation_1-error:0.292008
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 20 rounds.
[1]	validation_0-error:0.292008	validation_1-error:0.292008


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=0, min_child_weight=1, missing=None, n_estimators=2,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [29]:
train_accuracy = results['validation_0']['error']

In [30]:
print(train_accuracy)

[0.292008, 0.292008]


In [31]:
1.0 - train_accuracy

TypeError: unsupported operand type(s) for -: 'float' and 'list'

In [1]:
train_acc = [1.0 - x for x in train_accuracy]

NameError: name 'train_accuracy' is not defined

In [25]:
train_acc

[0.8312120000000001, 0.835566]